In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('/kaggle/input/hack-chunk-desc/train_segments.csv')

In [3]:
df

,Advertisement ID,Segment_num
0,1831845,16
1,4467135,11
2,2285593,0
3,3825366,0
4,2313008,0
...,...,...
17735,4299761,1
17736,2298577,0
17737,4018035,1
17738,3922868,0


In [4]:
segments = pd.read_excel('/kaggle/input/hack-chunk-desc/segment_dict.xlsx')

In [5]:
import os

all_files = []
base_path = '/kaggle/input/hack-chunk-'

for i in ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10-stranger', '11', '12-stranger', '13', '14', '15', '16', '17'):
    folder_path = f"{base_path}{i}"
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.join(root, file)
            all_files.append(full_path)

In [6]:
id_to_path = {}
for file_path in all_files:
    file_id = int(file_path.split('/')[-1].split('.')[0])
    id_to_path[file_id] = file_path

def get_path(ad_id):
    return id_to_path.get(ad_id, None)

In [7]:
df['path'] = df['Advertisement ID'].apply(get_path)

In [8]:
df

,Advertisement ID,Segment_num,path
0,1831845,16,/kaggle/input/hack-chunk-2/2/1831845.mp4
1,4467135,11,/kaggle/input/hack-chunk-16/16/4467135.avi
2,2285593,0,/kaggle/input/hack-chunk-5/5/2285593.wav
3,3825366,0,/kaggle/input/hack-chunk-8/8/3825366.avi
4,2313008,0,/kaggle/input/hack-chunk-6/6/2313008.wav
...,...,...,...
17735,4299761,1,/kaggle/input/hack-chunk-16/16/4299761.avi
17736,2298577,0,/kaggle/input/hack-chunk-5/5/2298577.wav
17737,4018035,1,/kaggle/input/hack-chunk-10-stranger/10/401803...
17738,3922868,0,/kaggle/input/hack-chunk-9/9/3922868.avi


In [9]:
df = df[~df['path'].str.endswith('.jpg')]
df = df[~df['path'].str.endswith('.wav')]

In [10]:
df

,Advertisement ID,Segment_num,path
0,1831845,16,/kaggle/input/hack-chunk-2/2/1831845.mp4
1,4467135,11,/kaggle/input/hack-chunk-16/16/4467135.avi
3,3825366,0,/kaggle/input/hack-chunk-8/8/3825366.avi
5,3959329,0,/kaggle/input/hack-chunk-9/9/3959329.avi
6,4321218,1,/kaggle/input/hack-chunk-16/16/4321218.avi
...,...,...,...
17734,4227145,1,/kaggle/input/hack-chunk-14/14/4227145.avi
17735,4299761,1,/kaggle/input/hack-chunk-16/16/4299761.avi
17737,4018035,1,/kaggle/input/hack-chunk-10-stranger/10/401803...
17738,3922868,0,/kaggle/input/hack-chunk-9/9/3922868.avi


In [11]:
video_path = df.path[0]

In [12]:
! pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 40.0 MB/s eta 0:00:00:00:0100:01


In [13]:
import av
import torch
import numpy as np
from transformers import AutoProcessor, AutoModel
from huggingface_hub import hf_hub_download

np.random.seed(42)

2024-05-17 21:28:28.450099: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 21:28:28.450238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 21:28:28.759457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]

    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)

    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)

    return indices

In [27]:
processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
model = AutoModel.from_pretrained("microsoft/xclip-base-patch32")

preprocessor_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/786M [00:00<?, ?B/s]

In [28]:
import cv2

def get_video_frames(video_path, frame_indices):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file '{video_path}'")
        return None
    
    frames = []
    for i in range(max(frame_indices) + 1):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)

    cap.release()
    return frames

In [29]:
def getFeatures(video_path):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    
    indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=total_frames)
    video = get_video_frames(video_path, indices)
    
#     container = av.open(video_path)
#     indices = sample_frame_indices(clip_len=8, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
#     video = read_video_pyav(container, indices)

    inputs = processor(videos=list(video), return_tensors="pt")
    video_features = model.get_video_features(**inputs)
    
    return video_features

In [50]:
%%time

getFeatures(video_path)

CPU times: user 1.55 s, sys: 121 ms, total: 1.68 s
Wall time: 1.01 s


tensor([[ 8.3028e-01,  5.4034e-01, -7.8905e-01,  2.1150e-01,  9.1496e-01,
          1.5247e+00,  1.0789e+00, -1.9332e-01, -2.3532e-01,  3.9884e-01,
         -1.5954e+00, -8.7492e-01,  9.6131e-01,  9.8322e-01, -5.4205e-01,
         -8.0296e-01, -5.1061e-02, -2.5501e+00, -1.1453e+00, -1.3867e+00,
          1.1263e+00, -3.3116e-01, -8.2960e-01, -1.2150e+00, -5.3932e-01,
         -7.6974e-01,  1.3144e+00, -1.3472e+00, -7.6896e-01,  5.8181e-01,
          7.1086e-01, -1.8527e+00,  4.2885e-01,  9.3011e-01,  5.8554e-02,
         -9.5838e-01, -1.0909e+00, -1.8135e+00, -9.7483e-04,  6.7175e-01,
         -8.3212e-01,  1.2790e+00,  6.4507e-01,  1.4285e-01,  6.2414e-01,
         -3.3871e-01,  4.4136e-01, -1.2956e-01, -1.0910e+00, -1.2935e+00,
          8.6570e-01, -5.9182e-01, -3.9237e-01,  2.4120e-01,  6.6143e-01,
         -3.3452e-01,  5.9115e-01,  9.5669e-01,  1.3541e+00,  4.8308e-01,
         -8.1268e-04, -1.9434e+00,  4.3933e-01,  9.6436e-02, -1.3582e-01,
         -1.4619e+00, -4.3829e-01, -9.

In [34]:
# torch.Size([1, 512])

In [51]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

X = df.drop('Segment_num', axis=1)
y = df['Segment_num']

X_train, X_test_val, y_train, y_train_val = train_test_split(X, y, test_size=0.05, stratify=y, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test_val, y_train_val, stratify=y_train_val, test_size=0.5, random_state=42)

rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)

df_train = pd.concat([X_res, y_res], axis=1)
df_val = pd.concat([X_valid, y_valid], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [52]:
len(df_train)

1596

In [53]:
len(df_val)

355

In [54]:
len(df_test)

355

In [55]:
def process_in_chunks(df, chunk_size, func):
    num_chunks = int(np.ceil(len(df) / chunk_size))
    results = []
    
    for i in range(num_chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(df))
        chunk = df.iloc[start:end].copy()
        chunk['embs'] = chunk['path'].apply(func)
        results.append(chunk)
        
    return pd.concat(results)

In [56]:
chunk_size = 50

In [ ]:
df_train_processed = process_in_chunks(df_train, chunk_size, getFeatures)

In [ ]:
df_val_processed = process_in_chunks(df_val, chunk_size, getFeatures)

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
1

In [ ]:
CatBoostModel_emb = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=5,
    use_best_model=True,
    loss_function='MultiClass',
    eval_metric='F1:use_weights=False',
    custom_metric='F1:use_weights=False',
    random_seed=42,
    embedding_features=['embs']
)

In [ ]:
CatBoostModel_emb.fit(
    df_train_processed['embs'], df_train_processed['Segment_num'],
    eval_set=(df_val_processed['embs'], df_val_processed['Segment_num']),
    plot=True
)

In [ ]:
CatBoostModel_emb.get_best_score()

In [ ]:
df_test_processed = process_in_chunks(df_test, chunk_size, getFeatures)